In [1]:
# Note: Cell is tagged to not show up in the mkdocs build
%load_ext autoreload
%autoreload 2

<div class="admonition abstract highlight">
    <p class="admonition-title">In short</p>
    <p>This tutorial shows how to create datasets with more advanced data-modalities through the .zarr format.</p>
</div>

## Pointer columns

Not all data might fit the tabular format, e.g. images or conformers. In that case, we have _pointer_ columns. Pointer columns do not contain the data itself, but rather store a reference to an external file from which the content can be loaded.

For now, we only support `.zarr` files as references. To learn more about `.zarr`, visit their documentation. Their [tutorial](https://zarr.readthedocs.io/en/stable/tutorial.html) specifically is a good read to better understand the main features. 

### Dummy example
For the sake of simplicity, let's assume we have just two datapoints. We will use this to demonstrate the idea behind pointer columns. 

In [2]:
import zarr
import platformdirs

import numpy as np
import datamol as dm
import pandas as pd

SAVE_DIR = dm.fs.join(platformdirs.user_cache_dir(appname="polaris-tutorials"), "002")

In [3]:
# Create two images and save them to a Zarr archive
base_path = dm.fs.join(SAVE_DIR, "data.zarr")
inp_col_name = "images"

images = np.random.random((2, 64, 64, 3))
root = zarr.open(base_path, "w")
root.array(inp_col_name, images)

<zarr.core.Array '/images' (2, 64, 64, 3) float64>

In [4]:
# For performance reasons, Polaris expects all data related to a column to be saved in a single Zarr array. 
# To index a specific element in that array, the pointer path can have a suffix to specify the index. 
train_path = f"{inp_col_name}#0"
test_path = f"{inp_col_name}#1"

In [5]:
tgt_col_name = "target"

table = pd.DataFrame(
    {
        inp_col_name: [train_path, test_path],  # Instead of the content, we specify paths
        tgt_col_name: np.random.random(2),
    }
)

In [6]:
from polaris.dataset import Dataset, ColumnAnnotation

dataset = Dataset(
    table=table,
    # To indicate that we are dealing with a pointer column here,
    # we need to annotate the column.
    annotations={"images": ColumnAnnotation(is_pointer=True)},
    # We also need to specify the path to the root of the Zarr archive
    zarr_root_path=base_path,
)

Note how the table does not contain the image data, but rather stores a path relative to the root of the Zarr. 

In [7]:
dataset.table.loc[0, "images"]

'images#0'

To load the data that is being pointed to, you can simply use the `Dataset.get_data()` utility method. 

In [8]:
dataset.get_data(col="images", row=0).shape

(64, 64, 3)

Creating a benchmark and the associated `Subset` objects will automatically do so! 

In [9]:
from polaris.benchmark import SingleTaskBenchmarkSpecification

benchmark = SingleTaskBenchmarkSpecification(
    dataset=dataset,
    input_cols=inp_col_name,
    target_cols=tgt_col_name,
    metrics="mean_absolute_error",
    split=([0], [1]),
)

In [10]:
train, test = benchmark.get_train_test_split()

for x, y in train:
    # At this point, the content is loaded from the path specified in the table
    print(x.shape)

(64, 64, 3)


## Creating datasets from `.zarr` arrays

While the above example works, creating the table with all paths from scratch is time-consuming when datasets get large. Instead, you can also automatically parse a Zarr archive into the expected tabular data structure. 

A Zarr archive can contain groups and arrays, where each group can again contain groups and arrays. Within Polaris, we expect the root to be a flat hierarchy that contains a single array per column.


### A single array for _all_ datapoints 

Polaris expects a flat zarr hierarchy, with a single array per pointer column: 
```
/
  column_a
```

Which will get parsed into a table like: 

| column_a          |
| ----------------- |
| column_a/array#1  |
| column_a/array#2  |
|       ...         |
| column_a/array#N  |

<div class="admonition info highlight">
    <p class="admonition-title">Note</p>
    <p>Notice the # suffix in the path, which indicates the index at which the data-point is stored within the big array. </p>
</div>

In [11]:
# Let's first create some dummy dataset with 1000 64x64 "images"
images = np.random.random((1000, 64, 64, 3))

In [12]:
path = dm.fs.join(SAVE_DIR, "zarr", "data.zarr")

with zarr.open(path, "w") as root:
    root.array(inp_col_name, images)

To create a dataset from a Zarr archive, we can use the convenience function `create_dataset_from_file()`.

In [13]:
from polaris.dataset import create_dataset_from_file

# Because Polaris might restructure the Zarr archive, 
# we need to specify a location to save the Zarr file to.
dataset = create_dataset_from_file(path, zarr_root_path=dm.fs.join(SAVE_DIR, "zarr", "processed.zarr"))

# The path refers to the original zarr directory we created in the above code block
dataset.table.iloc[0][inp_col_name]

'images#0'

In [14]:
dataset.get_data(col=inp_col_name, row=0).shape

(64, 64, 3)

## Saving the dataset

We can still easily save the dataset. All the pointer columns will be automatically updated. 

In [15]:
savedir = dm.fs.join(SAVE_DIR, "json")
json_path = dataset.to_json(savedir)

In [16]:
fs = dm.fs.get_mapper(path).fs
fs.ls(SAVE_DIR)

['/home/cas/.cache/polaris-tutorials/002/zarr',
 '/home/cas/.cache/polaris-tutorials/002/json',
 '/home/cas/.cache/polaris-tutorials/002/data.zarr']

Besides the `table.parquet` and `dataset.yaml`, we can now also see a `data` folder which stores the content for the additional content from the pointer columns.

## Load the dataset

In [17]:
Dataset.from_json(json_path)

{
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "default_adapters": {},
  "zarr_root_path": "/home/cas/.cache/polaris-tutorials/002/json/data.zarr",
  "md5sum": "5488b4909fd67d3208624288e720e1b8",
  "readme": "",
  "annotations": {
    "images": {
      "is_pointer": true,
      "modality": "UNKNOWN",
      "description": null,
      "user_attributes": {},
      "dtype": "object"
    }
  },
  "source": null,
  "license": null,
  "curation_reference": null,
  "cache_dir": "/home/cas/.cache/polaris/datasets/None/5488b4909fd67d3208624288e720e1b8",
  "artifact_id": null,
  "n_rows": 1000,
  "n_columns": 1
}

The End. 